# Property calculation with SAFT-VR-Mie EoS

First it is necessary to import the ``component`` and ``mixture`` classes and the SAFT-VR-Mie equation of state (``saftvrmie``).

In [1]:
import numpy as np
from sgtpy import component, mixture, saftvrmie

First, a component is defined with the ``component`` class function, then a mixture is set with the ``mixture class``. Finally, the ``eos`` object is created with the component and the ``saftvrmie`` function. 

The ``eos`` object includes the methods to evaluate properties from the equation of state, such as densities, pressure, fugacity coefficients, chemical potential and some thermal derived properties (residual entropy, residual enthalpy, residual heat capacities and speed of sound).

**warning:** thermal derived properties are computed with numerical derivatives using $O(h^4)$ approximation.

In the case of coarse-grained non-associating fluid mixtures, the ``eos`` object allows to compute the [influence parameter for SGT](https://aiche.onlinelibrary.wiley.com/doi/full/10.1002/aic.15190). This is done with the following correlation:

$$ \sqrt{\frac{c_{ii}}{N_{av}^2 \epsilon_{ii} \sigma_{ii}^5}} \left[0.12008 + 2.21979 \alpha_i \right]$$

$$ \alpha_i = \left[ \frac{\lambda_r}{\lambda_r - \lambda_a} \left( \frac{\lambda_r}{\lambda_a}\right)^{\frac{\lambda_a}{\lambda_r - \lambda_a}}\right] \left[\frac{1}{\lambda_a - 3} - \frac{1}{\lambda_r - 3} \right]$$

The ``eos.cii_correlation`` method is shown below for a mixture of methane and dodecane.

In [2]:
methane = component('methane', ms = 1.0, sigma = 3.752 , eps = 170.75,
                    lambda_r = 16.39, lambda_a = 6.)
dodecane = component('dodecane', ms = 4.0, sigma = 4.351 , eps = 378.56,
                    lambda_r = 18.41, lambda_a = 6.)
# interaction parameter optimized from phase equilibria data
kij = -0.02199102576365056
Kij = np.array([[0, kij], [kij, 0]])
mix = mixture(methane, dodecane)
# or
mix = methane + dodecane

mix.kij_saft(Kij)
eos = saftvrmie(mix)
eos.cii_correlation(overwrite = True)

array([1.92075094e-20, 1.27211926e-18])

Mixtures calculations with SAFT-VR-Mie will be exemplified for the mixture of Hexane, Ethanol and CPME.

First, the components are defined. Then a mixture and it's interactions are set up (including induced association).

In [3]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20, Mw=46.07)

cpme = component('cpme', ms =  2.32521144, sigma = 4.13606074, eps = 343.91193798, lambda_r = 14.15484877, 
                 lambda_a = 6.0, npol = 1.91990385,mupol = 1.27, sites=[0,0,1], cii = 3.5213681817448466e-19, Mw=100.16)

hexane = component('hexane', ms = 1.96720036, sigma = 4.54762477, eps = 377.60127994, 
                   lambda_r = 18.41193194, npol = 0., cii = 3.581510586936205e-19, Mw=86.18 )

# creating mixture
mix = mixture(hexane, ethanol)
# adding a component
mix.add_component(cpme)

# or 
mix = hexane + ethanol + cpme

# setting kij corrections
k12 = 0.011818492037463553
k13 = 0.0008700151297528677
k23 = 0.01015194
Kij = np.array([[0., k12, k13], [k12, 0., k23], [k13, k23, 0.]])
mix.kij_saft(Kij)

eos = saftvrmie(mix)

# induced association set up 
rc = 2.23153033
eos.eABij[1,2] = ethanol.eAB / 2
eos.eABij[2,1] = ethanol.eAB / 2
eos.rcij[1,2] = rc * 1e-10
eos.rcij[2,1] = rc * 1e-10

The density of the mixture is computed with the ``eos.density`` method. It requires the composition, temperature (K), pressure (Pa) and the aggregation state (``'L'`` for liquid phase or ``'V'`` for vapor phase).

When no initial guess has been provided. Topliss's method is used to initialize the density calculation.

In [4]:
T = 350.  # K
P = 1e5  # Pa
x = np.array([0.1, 0.3, 0.6])
# computed density in mol/m3
eos.density(x, T, P, 'L'), eos.density(x, T, P, 'V')

(9311.833947244266, 35.4581485427012)

Optionally, you can provide an initial guess to start computing the density. This is done with the ``rho0`` parameter.

In this case, Newton's method is used to solve the density.

In [5]:
T = 350.  # K
P = 1e5  # Pa
x = np.array([0.1, 0.3, 0.6])
# computed density in mol/m3
rho0 = 0.9*9311.833947244266
eos.density(x, T, P, 'L', rho0=rho0)

9311.833947244262

The pressure of the fluid mixture can be computed at given composition, molar density (mol/m3) and temperature (K) using the ``eos.pressure`` method.

In [6]:
T = 350.  # K
x = np.array([0.1, 0.3, 0.6])
rhol = 9311.833947244266  # mol/m3
rhov = 35.4581485427012  # mol/m3
#computed pressure in Pa
eos.pressure(x, rhol, T), eos.pressure(x, rhov, T) 

(99999.99999583118, 99999.99999968821)

The effective fugacity coefficients can be computed at given composition, temperature (K), pressure (Pa) and aggregation state using the ``eos.logfugef`` method. This function returns the natural logarithm of the fugacity coefficients and the computed volume root.

The fugacity of the mixture can be computed with the ``eos.logfugmix`` method. It works similarly to the previous method. 

The chemical potential are computed at given  density vector ($\rho_i = x_i \rho $) (mol/m3) and temperature (K).

In [7]:
T = 350.  # K
P = 1e5  # Pa
x = np.array([0.1, 0.3, 0.6])

lnphi, v = eos.logfugef(x, T, P, 'L')
lnphimix, v = eos.logfugmix(x, T, P, 'L')

print('Partial molar property : ', np.allclose(lnphimix, np.dot(lnphi, x)))


rholv = x/v   # mol/m3
# chemical potential divided by RT
eos.muad(rholv, T)

Partial molar property :  True


array([-1.25361318, -0.45605435, -1.03181168])

The ``eos`` object also includes the calculation of some thermal derived properties such as residual entropy (``eos.EntropyR``), residual enthalpy (``eos.EnthalpyR``), residual isochoric heat capacity (``eos.CvR``), residual isobaric heat capacity (``eos.CpR``).

For the speed of sound calculation (``eos.speed_sound``) the ideal gas heat capacities are required, in the example, the isochoric and isobaric ideal gas contributions are set to $3R/2$ and $5R/2$, respectively. Better values of ideal gas heat capacities contribution can be found from empirical correlations, such as the provided by DIPPR 801.

In [8]:
Sr = eos.EntropyR(x, T, P, 'L')
Hr = eos.EnthalpyR(x, T, P, 'L')
Cpr = eos.CpR(x, T, P, 'L')
Cvr = eos.CvR(x, 1./v, T)

R = 8.314 # J / mol K
CvId = 3*R/2
CpId = 5*R/2
w = eos.speed_sound(x, T, P, 'L', CvId=CvId, CpId=CpId)

print('Residual Entropy : ', Sr, 'J / mol K')
print('Residual Enthalpy : ', Hr, 'J / mol')
print('Residual isochoric heat capacity : ', Cvr, 'J / mol K')
print('Residual isobaric heat capacity : ', Cpr, 'J / mol K')
print('Speed of sound : ', w, 'm / s')

Residual Entropy :  -96.33130425660885 J / mol K
Residual Enthalpy :  -34450.632081267235 J / mol
Residual isochoric heat capacity :  27.42475894782458 J / mol K
Residual isobaric heat capacity :  58.62707720701845 J / mol K
Speed of sound :  1189.4081604852809 m / s


---
For further information about each method check out the documentation running: ``eos.function?``